In [ ]:
!pip install "numpy<2" scipy pandas scikit-learn matplotlib seaborn numba==0.60.* pyldavis gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 20.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you ha

In [ ]:
!pip uninstall -y thinc spacy fastai raft-dask-cu12 dask-cuda cudf-cu12 \
                distributed-ucxx-cu12 >/dev/null
##!!!!!!!!!!---Please--restart the code-execution program--restart the session------this step is very important!!!!!!!!!!!!!

In [ ]:
import numpy, scipy, gensim
print(numpy.__version__, scipy.__version__, gensim.__version__)
## 1.26.4//1.13.1//4.3.3

1.26.4 1.13.1 4.3.3


In [ ]:
!pip install -q kagglehub
import kagglehub, pathlib
dataset_path = kagglehub.dataset_download("vbmokin/nlp-with-disaster-tweets-cleaning-data")
print("Data has been download to", dataset_path)


100%|██████████| 1.00M/1.00M [00:00<00:00, 1.45MB/s]

Extracting files...
数据已下载到： /root/.cache/kagglehub/datasets/vbmokin/nlp-with-disaster-tweets-cleaning-data/versions/2


In [ ]:
import pandas as pd, pathlib
data_dir = pathlib.Path(dataset_path)
df = pd.read_csv(data_dir/'train_data_cleaning.csv')
print(df.head())


   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this  # earthquake...   
1   4     NaN      NaN           Forest fire near La Ronge Sask .  Canada   
2   5     NaN      NaN  All residents asked to  ' shelter in place '  ...   
3   6     NaN      NaN  13,000 people receive  # wildfires evacuation ...   
4   7     NaN      NaN  Just got sent this photo from Ruby  # Alaska a...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  


In [ ]:
import re, string, nltk
from nltk.tokenize import TweetTokenizer
from gensim.models.phrases import Phrases, Phraser

# 1) Prepare stopwords and TweetTokenizer
nltk.download('stopwords')                                              # :contentReference[oaicite:2]{index=2}
STOP = set(nltk.corpus.stopwords.words('english'))
EXTRA_STOP = {"iphone", "time_utc", "utc"}                              # :contentReference[oaicite:3]{index=3}
tt = TweetTokenizer(reduce_len=True, strip_handles=True)                # :contentReference[oaicite:4]{index=4}

def clean(text: str) -> list[str]:
    text = re.sub(r"http\S+", " ", text.lower())                        # Remove URLs
    tokens = [t for t in tt.tokenize(text)                              # Tweet-specific tokenizer
              if t.isalpha() and t not in STOP | EXTRA_STOP and len(t) > 2]
    return tokens

# 2) Apply the cleaning function
sentences_tokens = df["text"].map(clean)

# 3) Train the bigram model
phrases = Phrases(sentences_tokens, min_count=3, threshold=5)           # :contentReference[oaicite:7]{index=7}
bigram  = Phraser(phrases)                                              # Obtain FrozenPhrases

# 4) ★ Use apply + lambda to invoke bigram ★
sentences = sentences_tokens.apply(lambda toks: bigram[toks])

print(sentences.head(3))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0     [deeds, reason, earthquake, may, allah, forgive]
1             [forest_fire, near, ronge, sask, canada]
2    [residents, asked, shelter, place, notified, o...
Name: text, dtype: object


In [ ]:
from gensim.downloader import load as api_load
from itertools import chain

# 1️⃣ Prepare two independent sentence sources
wikitext = api_load('text8')       # gensim corpus object that can be iterated multiple times
disaster_sents = list(sentences)   # Materialize if desired, or define a similar Iterable class

# 2️⃣ Define an Iterable Class
class CombinedCorpus:
    def __iter__(self):
        # Each for-loop restarts from the beginning
        for sent in wikitext:
            yield sent
        for sent in disaster_sents:
            yield sent

corpus = CombinedCorpus()

# 3️⃣ Build the vocabulary and train
model = Word2Vec(
    vector_size=300, window=5, sg=1,
    min_count=2, workers=4
)
model.build_vocab(corpus)
model.train(
    corpus,
    total_examples=model.corpus_count,
    epochs=10
)


(127500673, 170645960)

In [ ]:
class Corpus:
  def __iter__(self):
    for tokens in sentences:
      if tokens:
        yield tokens

corpus = Corpus()


In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(
        sentences = combined_corpus,     # Any iterable of token lists
        vector_size = 250,      # Dimensionality of the word vectors
        window = 20,        # Context window size
        sg = 1,            # 1 = Skip-Gram, 0 = CBOW
        negative = 15,
        sample = 0.0001,        # 1=Skip‑Gram, 0=CBOW
        min_count = 2,        # Minimum frequency threshold
        workers = 4,         # Number of worker threads
        epochs = 25)       # Number of training epochs


In [ ]:
model_path = "/content/word2vec.bin"
model.wv.save_word2vec_format(model_path, binary=True)
print("model has been saved", model_path)


模型已保存： /content/word2vec.bin


In [ ]:
print("🌍 earthquake →", model.wv.most_similar("earthquake",  topn=10))
print("🔥 wildfire   →", model.wv.most_similar("wildfire",    topn=10))
print("🌊 flood      →", model.wv.most_similar("flood",       topn=5))
print("🌀 tornado    →", model.wv.most_similar("tornado",     topn=5))


🌍 earthquake → [('earthquakes', 0.6003329157829285), ('hanshin', 0.587181806564331), ('prieta', 0.573293924331665), ('epicenters', 0.5732309222221375), ('sylmar', 0.5678456425666809), ('tangshan', 0.5595299005508423), ('chuetsu', 0.5483529567718506), ('sismo', 0.5426573157310486), ('ancash', 0.5376684665679932), ('nisqually', 0.5360806584358215)]
🔥 wildfire   → [('northern_california', 0.7010824084281921), ('evacuation_order', 0.647872805595398), ('roosevelt_wash', 0.6472470760345459), ('fire_update', 0.6456758975982666), ('parker_ridge', 0.6453662514686584), ('wildfires', 0.6366869807243347), ('route_complex', 0.6360307335853577), ('idaho_fire', 0.6274527311325073), ('feels_like', 0.6195122003555298), ('alberta_storm', 0.6189935803413391)]
🌊 flood      → [('flooding', 0.4846715033054352), ('johnstown', 0.45947974920272827), ('turbid', 0.4534183144569397), ('spillway', 0.4429597854614258), ('floods', 0.44228100776672363)]
🌀 tornado    → [('panavia', 0.5671097636222839), ('hailstorm', 0

In [ ]:
print("🥫 food      →", model.wv.most_similar("food",topn=10))
print("🏠 shelter   →", model.wv.most_similar("shelter",topn=10))
print("💊 medicine  →", model.wv.most_similar("medicine",topn=10))
print("🚰 water     →", model.wv.most_similar("water",topn=10))


🥫 food      → [('foods', 0.5939129590988159), ('fishmeal', 0.5818348526954651), ('starchy', 0.55527663230896), ('shellfish', 0.5474386811256409), ('zenkimchi', 0.5434109568595886), ('wfp', 0.5426414608955383), ('foodstuffs', 0.5420812964439392), ('perishable', 0.5416118502616882), ('meat', 0.5390897393226624), ('dollies', 0.5384724736213684)]
🏠 shelter   → [('inclemency', 0.5569344162940979), ('sheltering', 0.524232804775238), ('shelters', 0.5167546272277832), ('clothe', 0.5145229697227478), ('sanitation', 0.49648115038871765), ('fallout', 0.4963216483592987), ('firewood', 0.4946717619895935), ('survivalring', 0.4943796992301941), ('torpedoed', 0.4852437376976013), ('incase', 0.48518747091293335)]
💊 medicine  → [('physiology', 0.6176894903182983), ('physiotherapy', 0.5692651867866516), ('medical', 0.5687744617462158), ('osteopathy', 0.5469571948051453), ('ayurveda', 0.5425676703453064), ('orthomolecular', 0.5366150140762329), ('nccam', 0.5224237442016602), ('pulmonology', 0.51965576410

In [ ]:
print("🇺🇸 california  →", model.wv.most_similar("california",  topn=5))
print("🇮🇩 indonesia   →", model.wv.most_similar("indonesia",   topn=5))

🇺🇸 california  → [('wild_fires', 0.9959452748298645), ('firefighters', 0.9938658475875854), ('thetawniest', 0.9937743544578552), ('federal', 0.9936322569847107), ('aria', 0.9935943484306335)]
🇮🇩 indonesia   → [('korea', 0.9993283152580261), ('fourth', 0.9993040561676025), ('deepest', 0.9992878437042236), ('cheap', 0.9992775321006775), ('condolences', 0.9992697834968567)]


In [ ]:
print("🌲 forest_fire  →", model.wv.most_similar("forest_fire", topn=5))
print("🛑 evacuation_order →", model.wv.most_similar("evacuation_order", topn=5))


🌲 forest_fire  → [('nuff', 0.7372389435768127), ('cuz', 0.720212996006012), ('campfire', 0.7192258834838867), ('tryna', 0.7001107931137085), ('gonna_get', 0.6970714926719666)]
🛑 evacuation_order → [('lifted_town', 0.8887737989425659), ('roosevelt_wash', 0.8177922964096069), ('cafire', 0.7164545059204102), ('alberta_storm', 0.7157350778579712), ('warning_oklahoma', 0.7067198753356934)]


In [ ]:
words = ["food","shelter","water"]
vectors = [model.wv[w] for w in words]
vectors_kv = model.wv.vectors_for_all(words)
print(vectors)


[array([ 0.52643585,  0.00090234, -0.04764531, -0.02034093, -0.02679369,
        0.02784539,  0.19168304,  0.08437032,  0.16393915,  0.21736465,
       -0.09588082,  0.1557533 ,  0.03872947,  0.06891293, -0.0483078 ,
       -0.06281126, -0.13731349, -0.19415681, -0.17996654, -0.1413677 ,
       -0.37603506,  0.00210785,  0.09007993,  0.01874966, -0.02688764,
       -0.16444133,  0.02312358, -0.10480254,  0.02538643, -0.02664514,
       -0.11262378,  0.0881897 , -0.23052819, -0.03469328, -0.1833586 ,
       -0.03414114, -0.22732723, -0.2658503 ,  0.11860419, -0.2553458 ,
       -0.18663155, -0.26120517,  0.15657973,  0.21255085,  0.08847817,
       -0.08787587, -0.10285738,  0.00947458,  0.1998685 , -0.11106337,
        0.09321106, -0.24255764,  0.15180176, -0.33091214,  0.0171075 ,
        0.1050768 ,  0.10136341,  0.04712922, -0.0819614 , -0.04710401,
        0.06536388,  0.07298957, -0.04432363,  0.16253936, -0.33551028,
       -0.16093381,  0.12457749,  0.02667357,  0.16941048, -0.1

In [ ]:
vec = model.wv['earthquake']
print(type(vec))     # <class 'numpy.ndarray'>
print(vec.shape)

<class 'numpy.ndarray'>
(250,)
